In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.functions import process_images

# Specify the folder path containing the images and annotations
data_path = '../data/'

# Specify the output folder path
output_folder = '../results/without_yolo/'

# process_images returns a dataframe containing the results
results = process_images(data_path, output_folder, highlighted_cars=True)
results.head(10)

Processing images...
Processed 23 images ✅


,Image File,Disabled parking spots count,Parking spots count,Cars count,Empty disabled parking spots count,Occupied disabled parking spots count,Empty parking spots count,Occupied parking spots count,Cars in transit or parked in non-parking spots
0,empty_set1_60m_1.jpg,20,70,1,20,0,69,1,0
1,empty_set1_60m_2.jpg,20,70,1,20,0,69,1,0
2,empty_set1_65m_1.jpg,22,77,0,22,0,77,0,0
3,empty_set1_70m_2.jpg,22,77,1,22,0,76,1,0
4,empty_set1_75m_1.jpg,22,77,0,22,0,77,0,0
5,empty_set2_30m_2.jpg,0,32,0,0,0,32,0,0
6,empty_set2_30m_3.jpg,0,40,0,0,0,40,0,0
7,empty_set2_70m_1.jpg,2,147,0,2,0,147,0,0
8,empty_set2_70m_2.jpg,4,145,0,4,0,145,0,0
9,occupied_set1_30m_1.jpg,8,16,14,8,0,2,14,0


In [3]:
total_images = len(results)
print(results['Cars count'].sum())

505


In [5]:
import torch
print(torch.cuda.is_available())

True
